Ref : https://python.langchain.com/docs/use_cases/sql/quickstart/

In [1]:
from openai import OpenAI
from sqlalchemy import create_engine
from langchain.chat_models import ChatOpenAI
from langchain_community.utilities import SQLDatabase
from langchain_community.agent_toolkits import create_sql_agent
from pyprojroot import here
import pandas as pd 
import warnings
warnings.filterwarnings("ignore")

<b>Load data from csv file into data frame</B>

In [ ]:
df = pd.read_csv(here("/Users/ashokarulsamy/Python Learning/itam/data/itam_files/itam_hw.csv"))
print(df.shape)
print(df.columns.tolist())
display(df.head(3))

<b>Load data from data frame into Database</b>

In [2]:
db_path = "/Users/ashokarulsamy/Python Learning/itam/data/sqlite_db/sqlite_itam.db"
db_path = f"sqlite:///{db_path}"
engine = create_engine(db_path)
#df.to_sql("itam_hw", engine, index=False)

<b>Create SQL DB object</b>

In [3]:
db = SQLDatabase(engine=engine)
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM itam_hw where assigned_to = 'Ashok Arulsamy'")

sqlite
['itam_hw']


"[('SW002', 'Source Software', 'Software', 'SAP', 'Version-82.2', '2022-11-06', '2027-04-17', 'Ashok Arulsamy', 'Active'), ('HW013', 'Majority Device', 'Hardware', 'Lenovo', 'Model-4924', '2022-04-22', '2029-10-02', 'Ashok Arulsamy', 'In Use'), ('HW024', 'Card Device', 'Hardware', 'Lenovo', 'Model-0757', '2022-12-27', '2028-05-04', 'Ashok Arulsamy', 'Retired'), ('HW053', 'Step Device', 'Hardware', 'Asus', 'Model-6226', '2022-10-14', '2029-05-23', 'Ashok Arulsamy', 'Available')]"

<b>Create LLM object</b>

In [4]:
llm = ChatOpenAI(
    api_key="",
    model_name="gpt-4o-mini",
    temperature=0.0,
    max_tokens=1000
)

<b>Create Agent</b>

In [5]:
agent_executor = create_sql_agent(llm, db=db, agent_type="openai-tools", verbose=True)

<b>Execute user query via agent</b>

In [6]:
agent_executor.invoke(
    {
        #"input": "How many assets assigned to Ashok Arulsamy"
        "input": "How many hardware assets are there?"
    }
)



> Entering new SQL Agent Executor chain...

Invoking: `sql_db_list_tables` with `{}`


0mitam_hw
Invoking: `sql_db_schema` with `{'table_names': 'itam_hw'}`



CREATE TABLE itam_hw (
	"Asset_ID" TEXT, 
	"Asset_Name" TEXT, 
	"Asset_Type" TEXT, 
	"Manufacturer" TEXT, 
	"Model" TEXT, 
	"Purchase_Date" TEXT, 
	"Warranty_Expiry" TEXT, 
	"Assigned_To" TEXT, 
	"Status" TEXT
)

/*
3 rows from itam_hw table:
Asset_ID	Asset_Name	Asset_Type	Manufacturer	Model	Purchase_Date	Warranty_Expiry	Assigned_To	Status
HW001	It Device	Hardware	Lenovo	Model-4915	2022-07-07	2027-12-29	William Jackson	Retired
SW002	Source Software	Software	SAP	Version-82.2	2022-11-06	2027-04-17	Ashok Arulsamy	Active
HW003	Green Device	Hardware	HP	Model-8953	2020-09-28	2027-03-16	Jennifer Adams	Retired

Invoking: `sql_db_query_checker` with `{'query': "SELECT COUNT(*) AS Total_Hardware_Assets FROM itam_hw WHERE Asset_Type = 'Hardware'"}`


```sql
SELECT COUNT(*) AS Total_Hardware_Assets FROM itam_hw WHERE Asset_Type = 'Hardwar

{'input': 'How many hardware assets are there?',
 'output': 'There are a total of 56 hardware assets.'}